In [1]:
using OptimalTransportDataIntegration
using Distributions
using DocStringExtensions
using Parameters
using Printf
using DataFrames
using Flux
#using PLots

In [30]:
params = DataParameters(
    nA = 1000,
    nB = 1000,
    mA = [0.0, 0.0, 0.0],
    mB = [1.0, 1.0, 0.0],
    covA = [1.0 0.2 0.2; 0.2 1.0 0.2; 0.2 0.2 1.0],
    covB = [1.0 0.2 0.2; 0.2 1.0 0.2; 0.2 0.2 1.0],
    aA = [1.0, 1.0, 1.0],
    aB = [1.0, 1.0, 1.0],
    r2 = 0.6
)
params = DataParameters(
    nA = 1000,
    nB = 1000,
    mA = [0.0],
    mB = [0.0],
    covA = ones(1,1),
    covB = ones(1,1),
    aA = [1.0],
    aB = [1.0],
    r2 = 1
) 
rng = ContinuousDataGenerator(params; scenario = 1)
data = generate(rng)

┌ Info: Categories in YA OrderedCollections.OrderedDict(1 => 254, 2 => 261, 3 => 248, 4 => 237)
└ @ OptimalTransportDataIntegration C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\src\generate_continuous_data.jl:143
┌ Info: Categories in ZA OrderedCollections.OrderedDict(1 => 345, 2 => 336, 3 => 319)
└ @ OptimalTransportDataIntegration C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\src\generate_continuous_data.jl:144
┌ Info: Categories in YB OrderedCollections.OrderedDict(1 => 291, 2 => 248, 3 => 204, 4 => 257)
└ @ OptimalTransportDataIntegration C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\src\generate_continuous_data.jl:145
┌ Info: Categories in ZB OrderedCollections.OrderedDict(1 => 365, 2 => 304, 3 => 331)
└ @ OptimalTransportDataIntegration C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\src\generate_continuous_data.jl:146


Row,X1,Y,Z,database
,Float64,Int64,Int64,Int64
1,0.0244108,3,2,1
2,-0.320536,2,2,1
3,0.184084,3,2,1
4,0.373145,3,2,1
5,1.68073,4,3,1
6,-0.253531,2,2,1
7,0.875784,4,3,1
8,0.0334005,3,2,1
9,1.4072,4,3,1


In [31]:
dba = subset(data, :database => ByRow(==(1)))
dbb = subset(data, :database => ByRow(==(2)))


cols = names(dba, r"^X")   

XA = transpose(Matrix(dba[:, cols]))
XB = transpose(Matrix(dbb[:, cols]))


1×1000 Matrix{Float64}:
 1.0  -1.0  -2.0  0.0  -2.0  1.0  -0.0  …  1.0  -2.0  -1.0  -1.0  -1.0  0.0

In [32]:
Ylevels = 1:4
Zlevels = 1:3
YA = Flux.onehotbatch(dba.Y, Ylevels)
ZB = Flux.onehotbatch(dbb.Z, Zlevels)



3×1000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  1  1  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  …  ⋅  ⋅  ⋅  ⋅  1  1  ⋅  1  1  1  1  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅     1  1  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1
 1  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  1  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅

In [59]:
import Pkg; Pkg.add("Distances")

   Resolving package versions...
      Compat entries added for 
  No Changes to `C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\Project.toml`
  No Changes to `C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\Manifest.toml`


In [34]:
using Distances
nA = size(dba, 1)
nB = size(dbb, 1)

wa = ones(nA) ./ nA
wb = ones(nB) ./ nB

C0 = pairwise(Euclidean(), XA, XB, dims = 2)

C = C0.^2 #./ maximum(C0)
display(C)
display(wa)
display(wb)

1000×1000 Matrix{Float64}:
 0.608293    0.71396     2.75202    0.110706     …  0.800691    0.0333699
 1.26535     0.250014    1.72653    0.459239        0.302353    0.278384
 0.384721    1.00929     3.30729    0.0299468       1.11194     0.000529056
 0.185931    1.42491     4.03068    0.000256327     1.54641     0.0275761
 0.768052    6.2564     10.9908     1.7519          6.50827     2.17162
 1.1191      0.321511    1.90711    0.372913     …  0.380531    0.212167
 0.00510384  2.87755     6.30158    0.268996        3.04917     0.447158
 0.594352    0.729232    2.78193    0.104804        0.81686     0.0301663
 0.363441    4.96289     9.25203    1.10264         5.18749     1.44028
 0.301369    1.15761     3.57166    0.0103558       1.26737     0.00233161
 ⋮                                               ⋱              
 3.84227     0.112412    0.229136   2.28906         0.0814684   1.85754
 0.711965    0.610137    2.54426    0.15727         0.690503    0.0607734
 2.26468     0.0144019   0

1000-element Vector{Float64}:
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 ⋮
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001

1000-element Vector{Float64}:
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 ⋮
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001
 0.001

In [61]:
import Pkg; Pkg.add("PythonOT")


   Resolving package versions...
    Updating git-repo `https://github.com/pnavaro/PythonOT.jl`
      Compat entries added for 
  No Changes to `C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\Project.toml`
  No Changes to `C:\Users\vgares\Documents\GitHub\OptimalTransportDataIntegration.jl\Manifest.toml`


In [36]:
using PythonOT
using Flux


show(sum(wa))
show(sum(wb))


1.00000000000000071.00000000000000070.968

In [41]:
G .= PythonOT.emd(wa, wb, C)

ZApred =  nA .* ZB * G'
YBpred =nB .* YA * G

show(accuracy(Flux.onecold(ZApred), dba.Z))

0.968

In [42]:
reg = 0.001
reg_m1 = 0.01
reg_m2 = 0.01
G .= PythonOT.mm_unbalanced(wa, wb, C, (reg_m1, reg_m2); reg = reg, div = "kl")
ZApred =  nA .* ZB * G'
YBpred =nB .* YA * G

show(accuracy(Flux.onecold(ZApred), dba.Z))

0.999

In [44]:
#version avec regul
reg=1000
G .= PythonOT.entropic_partial_wasserstein(wa, wb, C,reg, m=0.8)
ZApred =  nA .* ZB * G'
YBpred =nB .* YA * G

show(accuracy(Flux.onecold(ZApred), dba.Z))

0.345

In [45]:
#version avec regul
reg=0.001
G .= PythonOT.entropic_partial_wasserstein(wa, wb, C,reg, m=0.8)
ZApred =  nA .* ZB * G'
YBpred =nB .* YA * G

show(accuracy(Flux.onecold(ZApred), dba.Z))

0.345

In [47]:
#version sans regul
G .= PythonOT.partial_wasserstein(wa, wb, C, m=1)
ZApred =  nA .* ZB * G'
YBpred =nB .* YA * G

show(accuracy(Flux.onecold(ZApred), dba.Z))

0.345